# RAFT IVF-PQ tutorial
In this tutorial you will learn to build IVF-PQ index and use it to search approximate nearest neighbors (ANN).

In [110]:
import cupy as cp
import numpy as np

In [111]:
# We'll need to load store some data in this tutorial
import os
import tempfile

WORK_FOLDER = os.path.join(tempfile.gettempdir(), 'raft_ivf_pq_tutorial')

if not os.path.exists(WORK_FOLDER):
   os.makedirs(WORK_FOLDER)
print("The index and data will be saved in", WORK_FOLDER)

The index and data will be saved in /tmp/raft_ivf_pq_tutorial


## Get the data
We're going to use the data from [ANN benchmarks website](ann-benchmarks.com),
which provides a few datasets in [HDF5 format](https://www.hdfgroup.org/solutions/hdf5/).

In [112]:
DATASET_URL = "http://ann-benchmarks.com/sift-128-euclidean.hdf5"
DATASET_FILENAME = DATASET_URL.split('/')[-1]

## download the dataset
import urllib.request
dataset_path = os.path.join(WORK_FOLDER, DATASET_FILENAME)
if not os.path.exists(dataset_path):
    urllib.request.urlretrieve(DATASET_URL, dataset_path)

## Load the dataset

In [113]:
import h5py
f = h5py.File(dataset_path, "r")

metric = f.attrs['distance']

dataset = cp.array(f['train'])
queries = cp.array(f['test'])
gt_neighbors = cp.array(f['neighbors'])
gt_distances = cp.array(f['distances'])

print(f"Loaded dataset of size {dataset.shape}; metric: '{metric}'.")
print(f"Number of test queries: {queries.shape[0]}")

Loaded dataset of size (1000000, 128); metric: 'euclidean'.
Number of test queries: 10000


## Build the index
Construction of the index generally consists of two phases: training (building the clusters) and filling-in (extending the index with data).
In the first phase, a balanced hierarchical k-means algorithm clusters the training data.
In the second phase, the new data is classified and added into the appropriate clusters in the index.
Hence, a user should call `ivf_pq.build` once and then possibly `ivf_pq.extend` several times.
Though for user convenience `ivf_pq.build` by default adds the whole training set into the index.

In [114]:
# RAFT's DeviceResources controls the GPU, cuda stream, memory policies etc.
# For now, we just create a default instance.
from pylibraft.common import DeviceResources
resources = DeviceResources()

In [115]:
from pylibraft.neighbors import ivf_pq
# First, we need to initialize the build/indexing parameters.
# One of the more important parameters is the product quantisation (PQ) dim.
# Effectively, this parameter says
#      "shrink the dataset to this dimensionality to reduce the index size".
# It must be not bigger than the dataset dim,
# and it should be divisible by 32 for better GPU performance.
pq_dim = 1
while pq_dim * 2 < dataset.shape[1]:
    pq_dim = pq_dim * 2
# We'll use the ANN-benchmarks-provided metric and sensible defaults for the rest of parameters.
index_params = ivf_pq.IndexParams(n_lists=1024, metric=metric, pq_dim=pq_dim)

In [116]:
%%time
## Build the index
# This function takes a row-major either numpy or cupy (GPU) array.
# Generally, it's a bit faster with GPU inputs, but the CPU version may come in handy
# if the whole dataset cannot fit into GPU memory (or even CPU RAM -- use mmap for that).
index = ivf_pq.build(index_params, dataset, handle=resources)
index

CPU times: user 3.89 s, sys: 752 ms, total: 4.64 s
Wall time: 4.65 s


Index(type=IVF-PQ, metric=euclidean, codebook=subspace, size=1000000, dim=128, pq_dim=64, pq_bits=8, n_lists=1024, rot_dim=128)

## Search
The search function returns the requested number `k` of (approximate) nearest neighbor in no particular order.
Besides the queries and `k`, the function can take a few more parameters to tweak the performance of the algorithm.
Again, these are passed via the struct with some sensible defaults.

In [117]:
k = 10
search_params = ivf_pq.SearchParams()

In [118]:
%%time
distances, neighbors = ivf_pq.search(search_params, index, queries, k, handle=resources)

CPU times: user 41.7 ms, sys: 1.73 ms, total: 43.4 ms
Wall time: 43.1 ms


In [119]:
# Let's get the results back to CPU.
# pylibraft functions are often asynchronous so the GPU needs to be explicitly synchronized
distances = cp.asarray(distances)
neighbors = cp.asarray(neighbors)
resources.sync()

In [120]:
## Check the quality of the prediction (recall)
def calc_recall(found_indicies, ground_truth):
    found_indicies = cp.asarray(found_indicies)
    bs, k = found_indicies.shape
    if bs != ground_truth.shape[0]:
        raise RuntimeError(
            "Batch sizes do not match {} vs {}".format(
                bs, ground_truth.shape[0])
        )
    if k > ground_truth.shape[1]:
        raise RuntimeError(
            "Not enough indicies in the ground truth ({} > {})".format(
                k, ground_truth.shape[1])
        )
    n = 0
    # Go over the batch
    for i in range(bs):
        # Note, ivf-pq does not guarantee the ordered input, hence the use of intersect1d
        n += cp.intersect1d(found_indicies[i, :k], ground_truth[i, :k]).size
    recall = n / found_indicies.size
    return recall

recall_first_try = calc_recall(neighbors, gt_neighbors)
print(f"Got recall = {recall_first_try} with the default parameters (k = {k}).")

Got recall = 0.85227 with the default parameters (k = 10).
